# AMMI Deep Natural Language Processing: Lab 1

## 0. Introduction

In this tutorial we will train neural networks on the bAbI tasks using ParlAI framework.  
This tutorial can be run both in google colab or on your computer.  
The solutions will be added during the lab [here](https://fburl.com/ammi_dnlp_lab1).  

We will cover the following:
0. Introduction
    - Introduction to ParlAI and installation
    - Introduction to the bAbI tasks
1. Exploring the data:
    - Compute some statistics (number of examples in train, valid, test, size of examples...)
    - Look at some examples
2. Choose the appropriate metrics
3. Baselines
    - Ranom baseline
    - Majority class baseline
    - Information retrieval baseline
4. More elaborate models
   - Generative model: Seq2Seq
   - Ranking model: Memory Network
5. To go further
    - Additional ideas to try if you want to dig deeper

### ParlAI
[ParlAI](https://github.com/facebookresearch/ParlAI/blob/master/README.md) (pronounced “par-lay”) is a framework for dialogue AI research, implemented in Python.

Its goal is to provide researchers:

* a unified framework for sharing, training and testing dialogue models
* many popular datasets available all in one place -- from open-domain chitchat to visual question answering.
* a wide set of reference models -- from retrieval baselines to Transformers.
* seamless integration of Amazon Mechanical Turk for data collection and human evaluation
* integration with Facebook Messenger to connect agents with humans in a chat interface

Documentation can be found [here](http://www.parl.ai/static/docs/), some of this tutorial is inspired from the ParlAI documentation so feel free to go back and forth between the notebook and the documentation.


### Setup the notebook
If using google colab, make sure to use TPU runtime by going to ***Runtime > Change runtime type > Hardware accelerator: TPU > Save***

### Install ParlAI

Start by installing ParlAI from github. The ParlAI folder will be located in the home directory at `~/ParlAI/`.  
*Note: In a jupyter notebook, you can run arbitrary bash commands by prefixing them with a question mark, example: `!echo "Hello World"`*

In [ ]:
# Remove `> /dev/null` to see the output of commands
!git clone https://github.com/facebookresearch/ParlAI.git ~/ParlAI  > /dev/null
!cd ~/ParlAI && git checkout 6bd0e58692b3fd3a13b5f654944525ac1b7cd8e3
!cd ~/ParlAI && python3 setup.py develop > /dev/null

Cloning into '/root/ParlAI'...
remote: Enumerating objects: 39179, done.
remote: Counting objects: 100% (430/430), done.
remote: Compressing objects: 100% (319/319), done.
remote: Total 39179 (delta 175), reused 309 (delta 105), pack-reused 38749
Receiving objects: 100% (39179/39179), 64.98 MiB | 27.93 MiB/s, done.
Resolving deltas: 100% (27915/27915), done.
Note: checking out '6bd0e58692b3fd3a13b5f654944525ac1b7cd8e3'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 6bd0e586 light paper link on webpage! (#1536)
zip_safe flag not set; analyzing archive contents...
  File "build/bdist.linux-x86_64/egg/websocket/server.py",

Most of the scripts that we will use in ParlAI are located in the `~/ParlAI/examples` directory.  
Let's have a first glance at the scripts available, we will come back to them later:

In [ ]:
!ls ~/ParlAI/examples/

base_train.py	       eval_model.py		 remote.py
build_dict.py	       extract_image_feature.py  seq2seq_train_babi.py
build_pytorch_data.py  interactive.py		 train_model.py
display_data.py        profile_train.py
display_model.py       README.md


### The bAbI tasks
Many datasets and tasks are included in ParlAI, we will focus on the bAbI tasks.
The bAbI tasks are 20 synthetic tasks that each test a unique aspect of text and reasoning, and hence test different capabilities of learning models from [Weston et al. ‘16](http://arxiv.org/abs/1502.05698).

---
**Question 0.**  
Open the bAbI [paper](https://arxiv.org/pdf/1502.05698.pdf) and read the abstract  and section: *"3 The Tasks"* (until paragraph **Two or Three Supporting Facts**,  included).  
- **0.a.** Explain in your own words the motivations behind these tasks (in 2-3 sentences).

*ANSWER HERE*

---

These tasks can be downloaded and used directly from ParlAI.  
We will focus on tasks 1, 2 and 3, see examples below:


**Task 1: Single Supporting Fact**  
Mary went to the bathroom.  
John moved to the hallway.  
Mary travelled to the office.  
Where is Mary?  
**Answer: office**  


**Task 2: Two Supporting Facts**  
John is in the playground.  
John picked up the football.  
Bob went to the kitchen.  
Where is the football?  
**Answer: playground**


**Task 3: Three Supporting Facts **  
John picked up the apple.  
John went to the office.  
John went to the kitchen.  
John dropped the apple.   
Where was the apple before the kitchen?  
**Answer: office**



## 1. Exploring the data

First we need to download the data, we will use the `build_dict.py` as a dummy task to download the data.

In [ ]:
# Download the data silently
!python ~/ParlAI/examples/build_dict.py --task babi:task1k:1 --dict-file /tmp/babi1.dict
# Print a few examples
!head -n 30 ~/ParlAI/data/bAbI/tasks_1-20_v1-2/en-valid-10k-nosf/qa1_train.txt

[ Main ParlAI Arguments: ] 
[  batchsize: 1 ]
[  datapath: /root/ParlAI/data ]
[  datatype: train ]
[  download_path: /root/ParlAI/downloads ]
[  hide_labels: False ]
[  image_mode: raw ]
[  multitask_weights: [1] ]
[  numthreads: 1 ]
[  show_advanced_args: False ]
[  task: babi:task1k:1 ]
[ ParlAI Model Arguments: ] 
[  dict_class: None ]
[  init_model: None ]
[  model: None ]
[  model_file: None ]
[ PytorchData Arguments: ] 
[  batch_length_range: 5 ]
[  batch_sort_cache_type: pop ]
[  batch_sort_field: text ]
[  numworkers: 4 ]
[  pytorch_context_length: -1 ]
[  pytorch_datapath: None ]
[  pytorch_include_labels: True ]
[  pytorch_preprocess: False ]
[  pytorch_teacher_batch_sort: False ]
[  pytorch_teacher_dataset: None ]
[  pytorch_teacher_task: None ]
[  shuffle: False ]
[ Dictionary Loop Arguments: ] 
[  dict_include_test: False ]
[  dict_include_valid: False ]
[  dict_maxexs: -1 ]
[  log_every_n_secs: 2 ]
[ Dictionary Arguments: ] 
[  bpe_debug: False ]
[  dict_endtoken: __end_

The bAbI tasks were downloaded in `~/ParlAI/data/bAbI/tasks_1-20_v1-2/en-valid-nosf/`

In bAbI the data is organised as follows:
- **Dialog turn**: A dialog turn is a single utterance / statement. Each line in the file corresponds to one dialog turn.   
  Example: *"John went to the office."*
- **Sample (question)**: Every few dialog turns, a question can be asked that the model has to answer, this consitute a sample.  The question is followed by its ground truth answer, separated by a tab.
  Example: *"Where is John? `<tab>` bathroom"*
- **Episode**: a sequence of ordered coherent dialog turns that are related to each other form an episode. Each new episode is independant of the others. Each line starts with the dialog turn number in the current episode.


---
**Question 1.**
- **1.a.** Look at the training file of task 1 (`~/ParlAI/data/bAbI/tasks_1-20_v1-2/en/qa1_train.txt`) and compute the following information:
  - Number of episodes
  - Number of  samples (questions)
  - Number of dialog turns per episode
  - How many different answers are there in the train set? How many times does each appear? (*hint: Use a python [counter](https://docs.python.org/3/library/collections.html#collections.Counter)*)
  - How many unique words appear in the training set? How many time does each appear? (*hint: Use the Counter `most_common()` method*)

*Print the answer in the following code cell*
  
  ---

In [ ]:
# # FILL THIS CELL
# from collections import Counter


# task_1_train_path = '/root/ParlAI/data/bAbI/tasks_1-20_v1-2/en-valid-10k-nosf/qa1_train.txt'
# with open(task_1_train_path, 'r') as f:
#     # FILL CODE HERE

In [ ]:
# SOLUTION
from collections import Counter


task_1_train_path = '/root/ParlAI/data/bAbI/tasks_1-20_v1-2/en-valid-10k-nosf/qa1_train.txt'
i = 0
n_episodes = 0
n_questions = 0
n_total_dialog_turns = 0
possible_answers = Counter()
possible_answers_dict = {}
vocabulary = Counter()
with open(task_1_train_path, 'r') as f:
    for line in f:
        # Each line starts with an integer giving the dialog turn in the current episode.
        # Each episode contains dialog turns with questions (with the answer next to it separated by a tab)
        line = line.strip('\n')
        dialog_turn = int(line.split(' ')[0])
        if dialog_turn == 1:
            n_episodes += 1
        # Remove the dialog turn number
        line = ' '.join(line.split(' ')[1:])
        fields = line.split('\t')
        if len(fields) > 1:
            n_questions += 1
            possible_answers.update([fields[1]])
        vocabulary.update(fields[0].split(' '))
        n_total_dialog_turns += 1
print(f'Number of episodes: {n_episodes}')
print(f'Number of questions: {n_questions}')
print(f'Number of dialog turns per episode: {n_total_dialog_turns/n_episodes}')
print(f'Possible answers: {possible_answers} ({len(possible_answers)})')
print(f'Accuracy of a random model: {1/len(possible_answers):.4f}')
print(f'Vocabulary size: {len(vocabulary)}')
print(f'Most common words: {vocabulary.most_common()}')

Number of episodes: 1800
Number of questions: 9000
Number of dialog turns per episode: 15.0
Possible answers: Counter({'bathroom': 1564, 'hallway': 1517, 'garden': 1508, 'bedroom': 1473, 'kitchen': 1471, 'office': 1467}) (6)
Accuracy of a random model: 0.1667
Vocabulary size: 24
Most common words: [('to', 18000), ('the', 18000), ('Where', 9000), ('is', 9000), ('', 9000), ('went', 7225), ('Mary', 4535), ('Sandra', 4502), ('John', 4484), ('Daniel', 4479), ('journeyed', 3620), ('travelled', 3582), ('back', 3581), ('moved', 3573), ('bathroom.', 3070), ('hallway.', 3045), ('garden.', 2982), ('kitchen.', 2981), ('office.', 2963), ('bedroom.', 2959), ('John?', 2299), ('Mary?', 2265), ('Sandra?', 2244), ('Daniel?', 2192)]



- **1.b.** Use the appropriate script from the `~/ParlAI/examples/` to take a quick look at examples of the first bAbI task.  
Does the number of episodes and examples fit what you computed before? (*hint: you can use the argument `--task babi:task1k:1` to select the first babi task*)

In [ ]:
# FILL THIS CELL
# SOLUTION
!python ~/ParlAI/examples/display_data.py --task babi:task10k:1

[ optional arguments: ] 
[  display_ignore_fields: agent_reply ]
[  max_display_len: 1000 ]
[  num_examples: 10 ]
[ Main ParlAI Arguments: ] 
[  batchsize: 1 ]
[  datapath: /root/ParlAI/data ]
[  datatype: train:stream ]
[  download_path: /root/ParlAI/downloads ]
[  hide_labels: False ]
[  image_mode: raw ]
[  multitask_weights: [1] ]
[  numthreads: 1 ]
[  show_advanced_args: False ]
[  task: babi:task10k:1 ]
[ ParlAI Model Arguments: ] 
[  dict_class: None ]
[  init_model: None ]
[  model: None ]
[  model_file: None ]
[ PytorchData Arguments: ] 
[  batch_length_range: 5 ]
[  batch_sort_cache_type: pop ]
[  batch_sort_field: text ]
[  numworkers: 4 ]
[  pytorch_context_length: -1 ]
[  pytorch_datapath: None ]
[  pytorch_include_labels: True ]
[  pytorch_preprocess: False ]
[  pytorch_teacher_batch_sort: False ]
[  pytorch_teacher_dataset: None ]
[  pytorch_teacher_task: None ]
[  shuffle: False ]
[ ParlAI Image Preprocessing Arguments: ] 
[  image_cropsize: 224 ]
[  image_size: 256 ]
[

**Answer 1.b** 
<font color='white'>
<ol>
Yes! the number of episodes and examples fit what I compute before (1800 episodes and 9000 examples).
</ol>
</font>

## 2. Metrics

The bAbI task 1 expects single word answers among a small set of possible answers.


---
**Question 2**  
- **2.a.** Which metrics do you think are appropriate for evaluating a model on this task?   
-  **2.b.**  What are their respective strengths?  
-  **2.c.** When do they fail? (find specific examples)  


*ANSWER HERE* 




---

<font color='white'>
Answer 2.a
<ol>
The metrics that I think are appropriate for evaluating a model on this task are: 
  <li> F1 </li>
  <li> accuracy </li>
  <li> recall </li>
  <li> BLEU-4 </li>
  <li> top-k accuracy </li>
</ol>
<font>

<font color='white'>
Answer 2.b
<ol>
  <li> Accuracy: this is perfect, exact, matching of the response, averaged across all examples in the dataset. </li>

  <li> BLEU-4: this is the BLEU score between the predicted response and the reference response. It is measured on tokenized text, and uses NLTK to compute it. </li>
  <li> F1: This is the Unigram F1 overlap between your text and the reference response. </li>
</ol>
</font>

<font color='white'>
Answer 2.c
<ol>
Accuracy fail for imbalance classification. This example match that problem: Total Observations = 1000, Fraudulent  Observations = 20 and Non Fraudulent Observations = 980. In that case top-k accuracy becomes better.
</ol>
</font>

## 3. Baseline



We now have a clearer idea of the data distribution and the metrics that we can use.  
The next step is to start solving the tasks with a simple baseline. This will allow us to compare more elaborate models agains this baseline.  
Here are a few classical baselines:
- **Random model**: The model answers randomly among the set of possible answers for each question
-  **Majority class**: The model always answers with the most frequent answer in the training set (majority class)

We are going to reimplement these own baselines.  
Implementing a new model in ParlAI is detailed in the [tutorial](http://parl.ai/static/docs/seq2seq_tutorial.html) but for our simple baselines, we will only need to inherit the [Agent](https://github.com/facebookresearch/ParlAI/blob/6d246842d3f4e941dd3806f3d9fa62f607d48f59/parlai/core/agents.py#L50) class and override the `act()` method.

---
**Question 3**  
- **3.a.** What would be the accuracy of a model that choses a random answer among the set of possible answers for each question? 

*ANSWER HERE*

---

*Note: the `%%writefile` magic command in jupyter writes the content of the cell to a file at the given path.*

<font color='white'>
Answer 3.a
<ol>
Assuming $n$ questions and $k$ set of possible answers, the accuracy of a random model is:

$$ accuracy = \frac{n}{k}*100 \% $$
</ol>
</font>
<font color='white'>
So hereby the accuracy of our model that choses a random answer among the set of possible answers for each question is 0.16.
</font>

In [ ]:
!mkdir -p ~/ParlAI/parlai/agents/baseline/
!touch ~/ParlAI/parlai/agents/baseline/random.py
!touch ~/ParlAI/parlai/agents/baseline/majorityclass.py

- **3.b.**  Design a baseline that answers a random word in the set of possible answer (run it multiple time to observe variance in results).

In [ ]:
# # FILL THIS CELL
# %%writefile ~/ParlAI/parlai/agents/baseline/random.py
# import random

# from parlai.core.torch_agent import Agent


# class RandomAgent(Agent):
  
#     def act(self):
#         # FILL CODE HERE

In [ ]:
# SOLUTION
%%writefile ~/ParlAI/parlai/agents/baseline/random.py
import random

from parlai.core.torch_agent import Agent


class RandomAgent(Agent):
  
    def act(self):
        if 'label_candidates' not in self.observation:
            return
        candidates = list(self.observation['label_candidates'])
        reply = {'text': candidates[random.randrange(len(candidates))]}
        return reply

Overwriting /root/ParlAI/parlai/agents/baseline/random.py


In [ ]:
!python ~/ParlAI/examples/eval_model.py -t babi:task10k:1 -m baseline/random | grep accuracy -A 1

{'exs': 1000, 'accuracy': 0.175, 'f1': 0.175, 'bleu': 1.75e-10}


**Multiple runing**
- {'exs': 1000, 'accuracy': 0.16, 'f1': 0.16, 'bleu': 1.6e-10}
- {'exs': 1000, 'accuracy': 0.173, 'f1': 0.173, 'bleu': 1.73e-10}
- {'exs': 1000, 'accuracy': 0.17, 'f1': 0.17, 'bleu': 1.7e-10}
- {'exs': 1000, 'accuracy': 0.171, 'f1': 0.171, 'bleu': 1.71e-10}
- {'exs': 1000, 'accuracy': 0.163, 'f1': 0.163, 'bleu': 1.63e-10}
- {'exs': 1000, 'accuracy': 0.173, 'f1': 0.173, 'bleu': 1.73e-10}
- {'exs': 1000, 'accuracy': 0.174, 'f1': 0.174, 'bleu': 1.74e-10}
- {'exs': 1000, 'accuracy': 0.171, 'f1': 0.171, 'bleu': 1.71e-10}
- {'exs': 1000, 'accuracy': 0.167, 'f1': 0.167, 'bleu': 1.67e-10}
- {'exs': 1000, 'accuracy': 0.175, 'f1': 0.175, 'bleu': 1.75e-10}

**For the random model, we have observed many variations of accuracy after runing each time.**

In [ ]:
!python ~/ParlAI/examples/display_model.py -t babi:task10k:1 -m baseline/random -n 10 

[ optional arguments: ] 
[  display_ignore_fields:  ]
[  num_examples: 10 ]
[ Main ParlAI Arguments: ] 
[  batchsize: 1 ]
[  datapath: /root/ParlAI/data ]
[  datatype: valid ]
[  download_path: /root/ParlAI/downloads ]
[  hide_labels: False ]
[  image_mode: raw ]
[  multitask_weights: [1] ]
[  numthreads: 1 ]
[  show_advanced_args: False ]
[  task: babi:task10k:1 ]
[ ParlAI Model Arguments: ] 
[  dict_class: None ]
[  init_model: None ]
[  model: baseline/random ]
[  model_file: None ]
[ ParlAI Image Preprocessing Arguments: ] 
[  image_cropsize: 224 ]
[  image_size: 256 ]
[creating task(s): babi:task10k:1]
[loading fbdialog data:/root/ParlAI/data/bAbI/tasks_1-20_v1-2/en-valid-10k-nosf/qa1_valid.txt]
[babi:task10k:1]: Sandra travelled to the office.
Sandra went to the bathroom.
Where is Sandra?
[label_candidates: bedroom|kitchen|bathroom|office|garden|...and 1 more]
[eval_labels: bathroom]
   hallway
~~
[babi:task10k:1]: Mary went to the bedroom.
Daniel moved to the hallway.
Where is S

- **3.c.**  Design a baseline that answers the most common answer every time (majority class baseline).

In [ ]:
# # FILL THIS CELL
# %%writefile ~/ParlAI/parlai/agents/baseline/majorityclass.py
# import random

# from parlai.core.torch_agent import Agent


# class MajorityclassAgent(Agent):
  
#     def act(self):
#         # FILL CODE HERE

Overwriting /root/ParlAI/parlai/agents/baseline/majorityclass.py


In [ ]:
# SOLUTION
%%writefile ~/ParlAI/parlai/agents/baseline/majorityclass.py
import random

from parlai.core.torch_agent import Agent


class MajorityclassAgent(Agent):
  
    def act(self):
        if 'label_candidates' not in self.observation:
            return
        reply = {'text': 'bathroom'}
        return reply

Overwriting /root/ParlAI/parlai/agents/baseline/majorityclass.py


In [ ]:
!python ~/ParlAI/examples/eval_model.py -t babi:task10k:1 -m baseline/majorityclass | grep accuracy -A 1

{'exs': 1000, 'accuracy': 0.169, 'f1': 0.169, 'bleu': 1.69e-10}


**Multiple runing:**
- {'exs': 1000, 'accuracy': 0.169, 'f1': 0.169, 'bleu': 1.69e-10}
- {'exs': 1000, 'accuracy': 0.169, 'f1': 0.169, 'bleu': 1.69e-10}
- {'exs': 1000, 'accuracy': 0.169, 'f1': 0.169, 'bleu': 1.69e-10}
- {'exs': 1000, 'accuracy': 0.169, 'f1': 0.169, 'bleu': 1.69e-10}
- {'exs': 1000, 'accuracy': 0.169, 'f1': 0.169, 'bleu': 1.69e-10}

**For the majority class, there is no variation of accuracy after each running each time.**

In [ ]:
!python ~/ParlAI/examples/display_model.py -t babi:task10k:1 -m baseline/majorityclass -n 10

[ optional arguments: ] 
[  display_ignore_fields:  ]
[  num_examples: 10 ]
[ Main ParlAI Arguments: ] 
[  batchsize: 1 ]
[  datapath: /root/ParlAI/data ]
[  datatype: valid ]
[  download_path: /root/ParlAI/downloads ]
[  hide_labels: False ]
[  image_mode: raw ]
[  multitask_weights: [1] ]
[  numthreads: 1 ]
[  show_advanced_args: False ]
[  task: babi:task10k:1 ]
[ ParlAI Model Arguments: ] 
[  dict_class: None ]
[  init_model: None ]
[  model: baseline/majorityclass ]
[  model_file: None ]
[ ParlAI Image Preprocessing Arguments: ] 
[  image_cropsize: 224 ]
[  image_size: 256 ]
[creating task(s): babi:task10k:1]
[loading fbdialog data:/root/ParlAI/data/bAbI/tasks_1-20_v1-2/en-valid-10k-nosf/qa1_valid.txt]
[babi:task10k:1]: Sandra travelled to the office.
Sandra went to the bathroom.
Where is Sandra?
[eval_labels: bathroom]
[label_candidates: bedroom|kitchen|office|bathroom|hallway|...and 1 more]
   bathroom
~~
[babi:task10k:1]: Mary went to the bedroom.
Daniel moved to the hallway.
W

---
- **3.d.**  In which cases would the majority class baseline be better than the random baseline?

*ANSWER HERE*

---

<font color='white'>
Answer 3.d

In the case of imbalance dataset, the majority class baseline would be better than the random baseline.
</font>

Another slightly more advanced baseline is implemented in ParlAI: the information retrieval baseline (`ir_baseline`)

---
- **3.e.** Look at the [implementation](https://github.com/facebookresearch/ParlAI/blob/53ea58acf389bffc79c85c43bcdd848eecdcecb4/parlai/agents/ir_baseline/ir_baseline.py#L211) of the IR baseline and explain in a few lines how it works (*hint: look at the following methods `act()` `rank_candidates()`  `score_match()`*)  

*ANSWER HERE*


---

<font color='white'>
Answer 3.e
<ol>
    <li> This is a ranking model that gives a ranking score for each word based on its frequency. The more frequent the word is, the less score it will have. </li>
    <li> Then, it's going to score each given sentence based the words within the sentence and also based on the length of the sentence.</li>
    <li> Then, it's goning to order the sentences based on the given score.</li>
    <li>Finally, it returns the candidate with the highest score from the dialog history. </li>
</ol>
</font>

- **3.f.** Use the IR baseline and compare its with one of your baselines (random and/or majority) on bAbI tasks 1, 2 and 3.  
    (*hint: you can use `!python ... -t babi:task1-k:{i+1}` syntax to substitute the task number in a bash command from jupyter*)


In [ ]:
# SOLUTION
for i in range(3):
    print(f'~ Task {i+1} ~')
    print('Majority class baseline:')
    !python ~/ParlAI/examples/eval_model.py -t babi:task10k:{i+1} -m baseline/majorityclass | grep accuracy
    print('IR baseline:')
    !python ~/ParlAI/examples/eval_model.py -t babi:task10k:{i+1} -m ir_baseline | grep accuracy

~ Task 1 ~
Majority class baseline:
{'exs': 1000, 'accuracy': 0.169, 'f1': 0.169, 'bleu': 1.69e-10}
IR baseline:
{'exs': 1000, 'accuracy': 0.465, 'f1': 0.465, 'hits@1': 0.465, 'hits@5': 0.961, 'hits@10': 1.0, 'hits@100': 1.0, 'bleu': 4.65e-10}
~ Task 2 ~
Majority class baseline:
{'exs': 1000, 'accuracy': 0.17, 'f1': 0.17, 'bleu': 1.7e-10}
IR baseline:
{'exs': 1000, 'accuracy': 0.284, 'f1': 0.284, 'hits@1': 0.284, 'hits@5': 0.9, 'hits@10': 1.0, 'hits@100': 1.0, 'bleu': 2.84e-10}
~ Task 3 ~
Majority class baseline:
{'exs': 1000, 'accuracy': 0.203, 'f1': 0.203, 'bleu': 2.03e-10}
IR baseline:
{'exs': 1000, 'accuracy': 0.132, 'f1': 0.132, 'hits@1': 0.132, 'hits@5': 0.836, 'hits@10': 1.0, 'hits@100': 1.0, 'bleu': 1.32e-10}


<font color='white'>
Answer 3.f
<ol>
  <li> For task 1 the information in the IR baseline is better than the one in majority class baseline.</li>
  <li> For task 2 the information in the IR baseline is better than the one in majority class baseline. </li>
  <li> For task 3 the information in the IR baseline is not better than the one in majority class baseline.</li>
</ol>
</font>

## 4. More elaborate models



We can now continue to more elaborate models and evaluate their performance in perspective to the baselines.
We will use the `~/ParlAI/examples/train_model.py` script. Let's first get a glance at its arguments:

In [ ]:
!python ~/ParlAI/examples/train_model.py --help

usage: train_model.py [-h] [-v] [-t TASK]
                      [-dt {train,train:stream,train:ordered,train:ordered:stream,train:stream:ordered,train:evalmode,train:evalmode:stream,train:evalmode:ordered,train:evalmode:ordered:stream,train:evalmode:stream:ordered,valid,valid:stream,test,test:stream}]
                      [-nt NUMTHREADS] [-bs BATCHSIZE] [-dp DATAPATH] [-m MODEL] [-mf MODEL_FILE] [-im INIT_MODEL] [-et EVALTASK]
                      [-eps NUM_EPOCHS] [-ttim MAX_TRAIN_TIME] [-vtim VALIDATION_EVERY_N_SECS] [-stim SAVE_EVERY_N_SECS]
                      [-sval SAVE_AFTER_VALID] [-veps VALIDATION_EVERY_N_EPOCHS] [-vp VALIDATION_PATIENCE]
                      [-vmt VALIDATION_METRIC] [-vmm {max,min}] [-pyt PYTORCH_TEACHER_TASK] [-pytd PYTORCH_TEACHER_DATASET]

Train a model

optional arguments:
  -h, --help
        show this help message and exit

Main ParlAI Arguments:
  -v, --show-advanced-args
        Show hidden command line options (advanced users only) (default: Fa

We can train two types of models:
- **Generative models**: The model generates an answer from its vocabulary.
- **Ranking models**: The model is given a list of possible answers and has to choose the correct answer. This is much easier for the model since the list of possible answers is often way smaller than the size of the vocabulary


### Generative model: seq2seq with attention

The generative model we are going to train is a sequence to sequence model with attention based on [Sustskever et al. 2014](https://arxiv.org/abs/1409.3215) and [Bahdanau et al. 2014](https://arxiv.org/abs/1409.0473).
      
- **4.a.** Briefly explain how attention works in sequence to sequence neural networks.
- **4.b.** Do you think attention is useful for the babi tasks? How would you verify it experimentally?

*ANSWER HERE*

---
- **4.c.** Train a seq2seq on bAbI task 1 (10k) and compare its results to the baselines.
   (*hint: for faster training use the following arguments `--batchsize 32 --numthreads 1 --num-epochs 5 --hiddensize 64 --embeddingsize 64 --numlayers 1 --decoder shared`)


<font color='white'>
Answer 4.a
<ol>
<br> How attention works in sequence to sequence neural networks ? </br>
    <li> First, the encoder-decoder architecture works as usual. There is some input sentence that has been encoded to a sentence vector that gets passed to the decoder. </li>
    <li> Then, the decoder uses this sentence vector with the start symbol to produce an output. </li>
    <li> This decoder output is used with all output vectors from encoder to compute the attention score. </li>
    <li> Then, we apply a Softmax function over the attention score to get the distribution. </li>
    <li> Finally, we are going to concatenate the context vector with the decoder output to calculate the translated word. </li>
</ol>
</font>

<font color='white'>
Answer 4.b
<ol>
Yes attention is very useful for the babi task. So to verify it, we can formulate several different tasks as question answering problems.
</font>

<font color='white'>
Answer 4.c
<ol>
On training, the results of seq2seq on bAbI task 1 (10k) is better compare to the baselines.
</ol>
</font>

In [ ]:
# FILL THIS CELL
# SOLUTION
!python ~/ParlAI/examples/train_model.py --task babi:task10k:1 --model seq2seq  --model-file /tmp/babi_s2s --batchsize 32 --numthreads 1 --num-epochs 5 --hiddensize 64 --embeddingsize 64 --numlayers 1 --decoder shared

[ Main ParlAI Arguments: ] 
[  batchsize: 32 ]
[  datapath: /root/ParlAI/data ]
[  datatype: train ]
[  download_path: /root/ParlAI/downloads ]
[  hide_labels: False ]
[  image_mode: raw ]
[  multitask_weights: [1] ]
[  numthreads: 1 ]
[  show_advanced_args: False ]
[  task: babi:task10k:1 ]
[ ParlAI Model Arguments: ] 
[  dict_class: parlai.core.dict:DictionaryAgent ]
[  init_model: None ]
[  model: seq2seq ]
[  model_file: /tmp/babi_s2s ]
[ Training Loop Arguments: ] 
[  dict_build_first: True ]
[  display_examples: False ]
[  eval_batchsize: None ]
[  evaltask: None ]
[  load_from_checkpoint: False ]
[  max_train_time: -1 ]
[  num_epochs: 5.0 ]
[  save_after_valid: False ]
[  save_every_n_secs: -1 ]
[  validation_cutoff: 1.0 ]
[  validation_every_n_epochs: -1 ]
[  validation_every_n_secs: -1 ]
[  validation_max_exs: -1 ]
[  validation_metric: accuracy ]
[  validation_metric_mode: None ]
[  validation_patience: 10 ]
[  validation_share_agent: False ]
[ Tensorboard Arguments: ] 
[  te

In [ ]:
!python ~/ParlAI/examples/display_model.py --task babi:task10k:1 --model seq2seq --model-file /tmp/babi_s2s

[ optional arguments: ] 
[  display_ignore_fields:  ]
[  num_examples: 10 ]
[ Main ParlAI Arguments: ] 
[  batchsize: 1 ]
[  datapath: /root/ParlAI/data ]
[  datatype: valid ]
[  download_path: /root/ParlAI/downloads ]
[  hide_labels: False ]
[  image_mode: raw ]
[  multitask_weights: [1] ]
[  numthreads: 1 ]
[  show_advanced_args: False ]
[  task: babi:task10k:1 ]
[ ParlAI Model Arguments: ] 
[  dict_class: parlai.core.dict:DictionaryAgent ]
[  init_model: None ]
[  model: seq2seq ]
[  model_file: /tmp/babi_s2s ]
[ ParlAI Image Preprocessing Arguments: ] 
[  image_cropsize: 224 ]
[  image_size: 256 ]
[ Seq2Seq Arguments: ] 
[  attention: none ]
[  attention_length: 48 ]
[  attention_time: post ]
[  bidirectional: False ]
[  decoder: same ]
[  dropout: 0.1 ]
[  embeddingsize: 128 ]
[  hiddensize: 128 ]
[  input_dropout: 0.0 ]
[  lookuptable: unique ]
[  numlayers: 2 ]
[  numsoftmax: 1 ]
[  rnn_class: lstm ]
[ Torch Generator Agent: ] 
[  beam_block_ngram: 0 ]
[  beam_dot_log: False ]
[

### Ranking model: memory network

We saw in the class that Memory Networks ([Sukhbaatar et al. 15'](https://papers.nips.cc/paper/5846-end-to-end-memory-networks.pdf)) rely on an explicit memory "database". this is especially adapted to tasks where a few useful memories are "hidden" among distractor memories.  
These type of networks worktherefore  especially well for the bAbI tasks by turning the previous dialog turns as memories and the question as the query.  
Here is an illustration of how a memory network work:

![Memory Network schema](https://raw.githubusercontent.com/louismartin/ammi-2019-bordes-DeepNLP/master/lab1/memory_network.png)


---
**Question 4**  
- **4.d.** Explain how hops work in a memory network (either with words or formulas using the notations of the above figure)
- **4.e.** How can a memory network be used to rank multiple candidates?  
  (*hint: you can look at the [implementation](https://github.com/facebookresearch/ParlAI/blob/6bd0e58692b3fd3a13b5f654944525ac1b7cd8e3/parlai/agents/memnn/modules.py#L22) of the memory network in ParlAI and especially the `_score()` method. Recall how the IR baseline worked.*)
  
*ANSWER HERE*
  
 
---

<font color='white'>
<b>Answer 4.d</b> The following is how one hop works:
<br>
<ol>
    <li>Compute the similarity between the question vector & the memory vectors using a dot product</li>
    <li>Apply a softmax over these simialrity scores to get probability</li>
    <li>Apply a softmax over these simialrity scores to get the weights as numbers between 0 and 1</li>
    <li>Compute the weighted sum of the original memory vectors using the newly-calculated weights.</li>
    <li>The last result is the new input vector</li>
</ol>
We will repeat that for the number of hops.
<br>
</font>

<font color='white'>
Answer 4.e
<ol> 
By using a dot product
</ol>
</font>


- **4.f.** Using the ParlAI implementation, train a memory network on bAbI tasks 1, 2 and 3 (10k) and compare its results with the baselines.  
   (*hint: use a 1 thread, a batch size of 32 and 5 epochs*)


In [ ]:
# # FILL CELL
# for i in range(3):
#     print(f'~ Task {i+1} ~')
#     # FILL CODE HERE

In [ ]:
# SOLUTION
for i in range(3):
    print(f'~ Task {i+1} ~')
    !python ~/ParlAI/examples/train_model.py -t babi:task10k:{i+1} -m memnn -mf /tmp/babi{i+1}_memnn -bs 32 -eps 5 | grep "'accuracy':"

~ Task 1 ~
Building dictionary: 100% 9.00k/9.00k [00:00<00:00, 23.1kex/s]
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/root/ParlAI/parlai/core/torch_ranker_agent.py:376: UserWarning: [ Executing train mode with provided inline set of candidates ]
  ''.format(mode)
/root/ParlAI/parlai/core/torch_ranker_agent.py:214: UserWarning: Some training metrics are omitted for speed. Set the flag `--train-predict` to calculate train metrics.
  "Some training metrics are omitted for speed. Set the flag "
/root/ParlAI/parlai/core/torch_ranker_agent.py:376: UserWarning: [ Executing eval mode with provided inline set of candidates ]
  ''.format(mode)
valid:{'exs': 1000, 'accuracy': 0.979, 'f1': 0.979, 'hits@1': 0.979, 'hits@5': 1.0, 'hits@10': 1.0, 'hits@100': 1.0, 'bleu': 9.79e-10, 'lr': 1, 'num_updates': 1407, 'examples': 1000, 'loss': 73.98, 'm

## 5. To go further

If you want to go further you can try to do the following:

- Retrieve and plot the attention of the memory network for the different hops along the memories.
- For the seq2seq model, can you plot the training loss? The validation loss? Both on the same plot?
- Can you show an example of overfitting?
- Adapt the seq2seq model for ranking using the [torch ranker tutorial](http://www.parl.ai/static/docs/tutorial_torch_ranker_agent.html)
- Try multitasking babi and squad, does it improve the performance? (this will require more GPU power than what is available in google colab)
- You can play around with other models and other tasks
- Try interfacing ParlAI with [messenger](http://www.parl.ai/static/docs/tutorial_messenger.html )